In [1]:
import os
import os.path as osp

import numpy as np

import open3d as o3d

import utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd_path = "data/kinth/pointcloud/1720509720.141261339.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = pcd.voxel_down_sample(voxel_size=0.01)
pcd = pcd.remove_radius_outlier(nb_points=40, radius=0.2)[0] # (pcd, new indexed from old)

points = np.asarray(pcd.points)
points = points - points.mean(axis=0)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"before voxelized: {len(points)}")
points, _, _, _ = utils.voxel_downsample(points, 0.015, use_avg=False)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"after voxelized: {len(points)}")

max, min: [2.95497558 2.56843204 2.21088221] [-3.28902462 -3.96956807 -0.72011782]
before voxelized: 9303
max, min: [6.2475 6.5325 2.9325] [0.0075 0.0075 0.0075]
after voxelized: 5410


# height filtration

In [3]:
pivot = utils.get_average_pivot(points[:, 2])
points = points[points[:, 2] < pivot * 1.75]

# covariance rad feature

In [4]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

search_radius = 3
feat_rad_list = None
feat_rad_list_dump_file = f"data/kinth/pickle/feat_rad_{search_radius:.2f}_{osp.basename(pcd_path).split('.')[0]}.pkl"

# if osp.exists(feat_rad_list_dump_file):
#     with open(feat_rad_list_dump_file, "rb") as f:
#         feat_rad_list = pickle.load(f)
# else:
    # 多进程处理子空间坐标几何特征计算
num_worker = 4
batch_size = (len(points) - 1) // num_worker + 1 # round up
feat_rad_list_con = [None] * num_worker

with ProcessPoolExecutor(max_workers=num_worker) as executor:
    future_dict = {
        executor.submit(
            utils.eigval_radius,
            points.copy(),
            (i * batch_size, min((i + 1) * batch_size, len(points))),
            search_radius
        ) : i for i in range(num_worker)
    }
    for future in as_completed(future_dict):
        feat_list, neighbor_num_record = future.result()
        feat_rad_list_con[future_dict[future]] = feat_list
        print((np.array(neighbor_num_record) < 3).astype(np.int32).sum())
feat_rad_list = np.concatenate(feat_rad_list_con, axis=0)
    
    # if not osp.exists(osp.dirname(feat_rad_list_dump_file)):
    #     os.makedirs(osp.dirname(feat_rad_list_dump_file), exist_ok=True)
    # with open(feat_rad_list_dump_file, "wb") as f:
    #     pickle.dump(feat_rad_list, f)

eigval progress: 100%|████████████████████████████████████████| 1171/1171 [00:00<00:00, 2354.43it/s]


0
0
0
0


In [5]:
print(feat_rad_list)
feat_rad_color = feat_rad_list
feat_rad_color = feat_rad_color / (feat_rad_color.sum(axis=1).reshape(-1, 1) + 1e-9)
feat_rad_color = (feat_rad_color * 255.0).astype(np.int32)
print(feat_rad_color)


[[0.36755292 0.43934023 0.        ]
 [0.33998143 0.48864653 0.        ]
 [0.36665716 0.44013585 0.        ]
 ...
 [0.31730542 0.50342649 0.        ]
 [0.20831217 0.6469246  0.        ]
 [0.28273236 0.5836782  0.        ]]
[[116 138   0]
 [104 150   0]
 [115 139   0]
 ...
 [ 98 156   0]
 [ 62 192   0]
 [ 83 171   0]]


In [6]:
utils.npy2ply(points, feat_rad_color, "data/output/sample_rad.ply")

# covariance vtl feature

In [7]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

search_radius = 2.0
feat_vtl_list = None
feat_vtl_list_dump_file = f"data/kinth/pickle/feat_vtl_{search_radius:.2f}_{osp.basename(pcd_path).split('.')[0]}.pkl"

# if osp.exists(feat_vtl_list_dump_file):
#     with open(feat_vtl_list_dump_file, "rb") as f:
#         feat_vtl_list = pickle.load(f)
# else:
    # 多进程处理子空间坐标几何特征计算
num_worker = 4
batch_size = (len(points) - 1) // num_worker + 1 # round up
feat_vtl_list_con = [None] * num_worker

with ProcessPoolExecutor(max_workers=num_worker) as executor:
    future_dict = {
        executor.submit(
            utils.eigval_vertic,
            points,
            (i * batch_size, min((i + 1) * batch_size, len(points))),
            search_radius
        ) : i for i in range(num_worker)
    }
    for future in as_completed(future_dict):
        feat_list, _ = future.result()
        feat_vtl_list_con[future_dict[future]] = feat_list
feat_vtl_list = np.concatenate(feat_vtl_list_con, axis=0)
    
    # if not osp.exists(osp.dirname(feat_vtl_list_dump_file)):
    #     os.makedirs(osp.dirname(feat_vtl_list_dump_file), exist_ok=True)
    # with open(feat_vtl_list_dump_file, "wb") as f:
    #     pickle.dump(feat_vtl_list, f)

eigval progress: 100%|████████████████████████████████████████| 1171/1171 [00:00<00:00, 3581.91it/s]


In [8]:
feat_vtl_color = (feat_vtl_list - feat_vtl_list.min(axis=1).reshape(-1,1) + 1e-3) / (feat_vtl_list.max(axis=1).reshape(-1, 1) + 1e-3)
feat_vtl_color = (feat_vtl_color * 255.0).astype(np.int32)

In [9]:
utils.npy2ply(points, feat_vtl_color, "data/output/sample_vtl.ply")